# Compute Embeddings for the WikiVoyage Data

In [20]:
import pandas as pd 
import numpy as np 
import re
import json
import os 
from sentence_transformers import SentenceTransformer
import pickle

### Computing embeddings for Wikivoyage Documents

In [21]:
wikivoyage_dir = "../../european-city-data/data-sources/wikivoyage/"
embeddings_dir = wikivoyage_dir + "/embeddings/"
data_dir = wikivoyage_dir + "cleaned/"
len(os.listdir(wikivoyage_dir))

6

In [22]:
cities = pd.read_csv("../city_abstracts_embeddings.csv")
cities.head()

,city,country,lat,lng,population,abstract,combined,embedding
0,Aalborg,Denmark,57.0500,9.9167,143598.0,"Aalborg is the largest city in North Jutland, ...","city: Aalborg, country: Denmark, population: 1...","[-0.0032697843853384256, 0.007246419321745634,..."
1,Adana,Turkey,37.0000,35.3213,1765981.0,Adana is a city on the Cilician Plains of cent...,"city: Adana, country: Turkey, population: 1765...","[-0.020492911338806152, 0.0039081997238099575,..."
2,Amsterdam,Netherlands,52.3728,4.8936,1459402.0,Amsterdam is the capital of the Netherlands. I...,"city: Amsterdam, country: Netherlands, populat...","[0.01612328179180622, -0.0028123168740421534, ..."
3,Ancona,Italy,43.6169,13.5167,100924.0,Ancona is the capital of the Italian region ca...,"city: Ancona, country: Italy, population: 1009...","[0.0012721708044409752, 0.0070460038259625435,..."
4,Ankara,Turkey,39.9300,32.8500,5503985.0,"Ankara is the capital of Turkey, central withi...","city: Ankara, country: Turkey, population: 550...","[0.006992552895098925, -0.0026167023461312056,..."


In [23]:
def compute_wikivoyage_doc_embeddings():
    wikivoyage_embeddings = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vector_dimension = model.get_sentence_embedding_dimension()

    for index, row in cities.iterrows():
        with open(data_dir + row['city'] + ".txt") as file:
            text = file.readlines()
            print("Computing embeddings for {0}".format(row['city']))
            embeddings = model.encode(text, show_progress_bar=True)

            file_name = embeddings_dir + row['city'] + ".pkl"

            with open(file_name, "wb") as fOut:
                pickle.dump({'text': text, 'embeddings': embeddings},fOut)

# compute_wikivoyage_doc_embeddings()

#         wikivoyage_embeddings.append({
#             'city': row['city'],
#             'country': row['country'],
#             'embedding': embeddings.tolist()
#         })
#         print("Added embeddings for {0}".format(row['city']))

### Computing Embeddings for Wikivoyage Listings

In [29]:
listings_dir = "../../european-city-data/data-sources/wikivoyage/listings/"

listings = pd.read_csv(listings_dir + "wikivoyage-listings-cleaned.csv")
listings.head()

,city,type,title,description
0,Aalborg,see,Aalborg Akvavit,Distillery.
1,Aalborg,see,Aalborghus Castle,A castle and garden from the 16th century.
2,Aalborg,see,Aalborg Tower,No description found.
3,Aalborg,see,Aalborg Townhall,Built in 1759.
4,Aalborg,see,Aalborg Zoo,"Cosy zoo with many ""classic"" zoo animals such ..."


In [36]:
def find_country(city):
    return cities[cities['city'] == city]['country'].values[0]

listings['country'] = listings['city'].apply(find_country)

In [40]:
combined = []
for i, row in listings.iterrows():
    combined.append(f"city: {row['city']}, country: {row['country']}, type: {row['type']}, title: {row['title']}, description: {row['description']}")

listings['combined'] = combined
listings.head()

,city,type,title,description,country,combined
0,Aalborg,see,Aalborg Akvavit,Distillery.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
1,Aalborg,see,Aalborghus Castle,A castle and garden from the 16th century.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
2,Aalborg,see,Aalborg Tower,No description found.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
3,Aalborg,see,Aalborg Townhall,Built in 1759.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
4,Aalborg,see,Aalborg Zoo,"Cosy zoo with many ""classic"" zoo animals such ...",Denmark,"city: Aalborg, country: Denmark, type: see, ti..."


In [49]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
vector_dimension = model.get_sentence_embedding_dimension()

In [51]:
listings

,city,type,title,description,country,combined
0,Aalborg,see,Aalborg Akvavit,Distillery.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
1,Aalborg,see,Aalborghus Castle,A castle and garden from the 16th century.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
2,Aalborg,see,Aalborg Tower,No description found.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
3,Aalborg,see,Aalborg Townhall,Built in 1759.,Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
4,Aalborg,see,Aalborg Zoo,"Cosy zoo with many ""classic"" zoo animals such ...",Denmark,"city: Aalborg, country: Denmark, type: see, ti..."
...,...,...,...,...,...,...
7163,Rivne,sleep,Marlen Hotel,Wi-Fi. 10 room / 21 bed.,Ukraine,"city: Rivne, country: Ukraine, type: sleep, ti..."
7164,Rivne,sleep,Feniks Hotel,No description found.,Ukraine,"city: Rivne, country: Ukraine, type: sleep, ti..."
7165,Rivne,sleep,Ukraine,70 r / 125 beds.,Ukraine,"city: Rivne, country: Ukraine, type: sleep, ti..."
7166,Rivne,sleep,Myr Hotel,18 rooms. / 28 beds.,Ukraine,"city: Rivne, country: Ukraine, type: sleep, ti..."


In [54]:
wv_embeddings = "../../european-city-data/data-sources/wikivoyage/listings/"

def compute_wv_listings_embeddings(df):
    # model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # vector_dimension = model.get_sentence_embedding_dimension()
    
    print("Computing embeddings")
    embeddings = []
    for i, row in df.iterrows():
        emb = model.encode(row['combined'], show_progress_bar=True).tolist()
        embeddings.append(emb)

    print("Finished computing embeddings for wikivoyage listings. Saving file...")
    df['embeddings'] = embeddings
    df.to_csv(wv_embeddings + "wikivoyage-listings-embeddings.csv")
    print("Finished saving file.")

compute_wv_listings_embeddings(listings)

Computing embeddings


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 59.87it/s]


Finished computing embeddings for wikivoyage listings. Saving file...
Finished saving file.


In [8]:
# wikivoyage = pd.DataFrame(wikivoyage_embeddings)

In [41]:
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# vector_dimension = model.get_sentence_embedding_dimension()

In [8]:
# from qdrant_client import QdrantClient

# client = QdrantClient(
#     url="https://cb8144d3-3bc4-4b48-9b8f-0d5162835b62.us-east4-0.gcp.cloud.qdrant.io:6333", 
#     api_key="Kxe4k8QDskHicKFj7DVMkLSHW1A7jApqX1nCN09jm1WPH9ycBVFO8Q",
# )

In [42]:
# from qdrant_client.models import VectorParams, Distance

# client.delete_collection(collection_name="wikivoyage")

# client.recreate_collection(
#    collection_name="wikivoyage",
#    vectors_config=VectorParams(size=vector_dimension, distance=Distance.COSINE, on_disk=True),
# )

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_94522/3473645509.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [43]:
# from qdrant_client.models import PointStruct # Import the PointStruct to store the vector and payload
# from tqdm import tqdm # Library to show the progress bar 

# # Populate collection with vectors using tqdm to show progress
# def insert_wikivoyage_listings():
#     for k, v in tqdm(listings.iterrows(), desc="Upserting articles", total=len(listings)):
#         try:
#             client.upsert(
#                 collection_name='wikivoyage',
#                 points=[
#                     PointStruct(
#                         id=k,
#                         vector=model.encode(v['combined'], show_progress_bar=True).tolist(),
#                         payload={
#                             'city': v['city'], 
#                             'country': v['country'],
#                             'type': v['type'],
#                             'title': v['title']
#                         }
#                     )
#                 ]
#             )
#         except Exception as e:
#             print(f"Failed to upsert row {k}: {v}")
#             print(f"Exception: {e}")

# # insert_wikivoyage_listings()

Batches: 100%|██████████| 1/1 [00:00<00:00, 45.68it/s]15:55,  5.77it/s]


Failed to upsert row 1657: city                                                    Debrecen
type                                                         see
title                                         Millenium fountain
description                                No description found.
country                                                  Hungary
combined       city: Debrecen, country: Hungary, type: see, t...
Name: 1657, dtype: object
Exception: Server disconnected without sending a response.


Upserting articles:  91%|█████████ | 6539/7168 [26:27<17:16,  1.65s/it]

Failed to upsert row 6538: city                                                    Valencia
type                                                       drink
title                                    Horchatería Els Sariers
description    A very large and popular horchatería with arti...
country                                                    Spain
combined       city: Valencia, country: Spain, type: drink, t...
Name: 6538, dtype: object
Exception: The read operation timed out


Upserting articles:  97%|█████████▋| 6984/7168 [28:51<08:09,  2.66s/it]

Failed to upsert row 6983: city                                                      Zurich
type                                                         eat
title                                                  Gambrinus
description    is a typical Swiss restaurant with good food a...
country                                              Switzerland
combined       city: Zurich, country: Switzerland, type: eat,...
Name: 6983, dtype: object
Exception: The read operation timed out


Upserting articles: 100%|██████████| 7168/7168 [29:50<00:00,  4.00it/s]


In [45]:
# import qdrant
def query_qdrant(query, vector_name='title', top_k=20):

    # Creates embedding vector from user query
    embedding = model.encode(query).tolist()
    
    query_results = client.search(
        collection_name='wikivoyage',
        query_vector=(
            embedding
        ),
        limit=top_k, 
        query_filter=None
    )
    
    return query_results

In [48]:
# query_results = query_qdrant('Museums in Europe')
# for i, article in enumerate(query_results):
#     print(f'{i + 1}. {article.payload["title"]}, City: {article.payload["city"]} (Score: {round(article.score, 3)})')

1. Museo Europeo degli Studenti, City: Bologna (Score: 0.64)
2. Staatliches Museum für Naturkunde Stuttgart, City: Stuttgart (Score: 0.637)
3. Musée gallo-romain de Fourvière, City: Lyon (Score: 0.631)
4. Musée Alsacien, City: Strasbourg (Score: 0.631)
5. Fine Arts Museum, City: Vitoria-Gasteiz (Score: 0.612)
6. Musée d'Art contemporain, City: Lyon (Score: 0.607)
7. Albertinum Museum, City: Dresden (Score: 0.601)
8. Musée du Cinema/Filmmuseum, City: Brussels (Score: 0.599)
9. Museu de la impremta i les Arts Gràfiques, City: Valencia (Score: 0.597)
10. National Museum of History, City: Sofia (Score: 0.595)
11. Thyssen-Bornemisza Museum of Art, City: Madrid (Score: 0.59)
12. Leonhardi Museum, City: Dresden (Score: 0.584)
13. Bibliotheca Wittockiana, City: Brussels (Score: 0.579)
14. Uzhhorod Castle, City: Uzhhorod (Score: 0.579)
15. Museum of Natural History, City: Lille (Score: 0.578)
16. Long Night of the Museums, City: Stuttgart (Score: 0.578)
17. Ottó Herman Museum, Arts museum, City